In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sys
import gc
import os
import glob
from pathlib import Path

current_dir = os.getcwd()
repo_root = os.path.abspath(os.path.join(current_dir, '..'))

if repo_root not in sys.path:
    sys.path.append(repo_root)

print(f"Repository Root: {repo_root}")

from dataloading.datasets import LadosDataset, StrantzaDataset
from models.mit_b2 import MIT_B2

device = "cuda" if torch.cuda.is_available() else "cpu" 
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

In [ ]:
DATASET_CHOICE = "dataset1"   #  "dataset1" or "dataset2"

DATASET_MAP = {
    "dataset1": {
        "name": "dataset1",
        "class": LadosDataset,
        "ckpt_imagenet": "Dataset1-FineTuned-ImageNet-Initialization",
        "ckpt_synth":   "Dataset1-FineTuned-Synthetic-Pretrained",
    },
    "dataset2": {
        "name": "dataset2",
        "class": StrantzaDataset,
        "ckpt_imagenet": "Dataset2-FineTuned-ImageNet-Initialization",
        "ckpt_synth":   "Dataset2-FineTuned-Synthetic-Pretrained",
    }
}

cfg = DATASET_MAP[DATASET_CHOICE]
print("Selected dataset:", cfg["name"])

In [ ]:
import os
import glob
import requests
from pathlib import Path

def find_checkpoint(fragment, osf_project_id=None):
    """
    Find checkpoint locally or download from OSF if not found.
    
    Args:
        fragment: String fragment to match checkpoint filename
    """
    ckpt_dir = os.path.join(repo_root, "output", "checkpoints")
    os.makedirs(ckpt_dir, exist_ok=True)
    
    # First, try to find locally
    ckpts = glob.glob(os.path.join(ckpt_dir, "*"))
    found = [c for c in ckpts if fragment.lower() in os.path.basename(c).lower()]
    
    if found:
        return max(found, key=os.path.getctime)
    
    # If not found locally and OSF project ID provided, try to download
    if osf_project_id:
        print(f"🔍 Checkpoint not found locally. Attempting to download from OSF...")
        downloaded_path = download_from_osf(fragment, ckpt_dir, osf_project_id)
        if downloaded_path:
            return downloaded_path
    
    print(f"No checkpoint matching: {fragment}")
    return None

def download_from_osf(fragment, ckpt_dir, osf_project_id):
    """
    Download checkpoint from OSF storage.
    
    Args:
        fragment: Filename fragment to search for
        ckpt_dir: Local directory to save checkpoint
        osf_project_id: OSF project ID
    """
    # OSF API endpoint to list files
    api_url = f"https://api.osf.io/v2/nodes/g364t/files/osfstorage/"
    
    try:
        # Get list of files in OSF project
        response = requests.get(api_url)
        response.raise_for_status()
        files = response.json()['data']
        
        # Find matching file
        matching_file = None
        for file in files:
            filename = file['attributes']['name']
            if fragment.lower() in filename.lower() and filename.endswith('.ckpt'):
                matching_file = file
                break
        
        if not matching_file:
            print(f" No checkpoint matching '{fragment}' found on OSF")
            return None
        
        # Get download URL
        download_url = matching_file['links']['download']
        filename = matching_file['attributes']['name']
        local_path = os.path.join(ckpt_dir, filename)
        
        print(f"Downloading {filename}...")
        
        # Download file with progress
        response = requests.get(download_url, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        block_size = 8192
        
        with open(local_path, 'wb') as f:
            downloaded = 0
            for chunk in response.iter_content(chunk_size=block_size):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    if total_size > 0:
                        percent = (downloaded / total_size) * 100
                        print(f"\r⬇️  Progress: {percent:.1f}%", end='')
        
        print(f"\nDownloaded: {filename}")
        return local_path
        
    except requests.exceptions.RequestException as e:
        print(f"Error downloading from OSF: {e}")
        return None

def load_model(ckpt_path):
    model = MIT_B2.load_from_checkpoint(ckpt_path)
    model.eval()
    return model.to(device)

def load_dataset(cfg):
    data_dir = os.path.join(repo_root, "data", cfg["name"])
    return cfg["class"](root_dir=data_dir, normalize_percentile=True)

In [ ]:
class GradCAMGenerator:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        self.hook_fwd = target_layer.register_forward_hook(self._save_act)
        self.hook_bwd = target_layer.register_full_backward_hook(self._save_grad)

    def remove_hooks(self):
        self.hook_fwd.remove()
        self.hook_bwd.remove()

    def _save_act(self, m, inp, out):
        if isinstance(out, (tuple,list)): out = out[0]
        self.activations = out

    def _save_grad(self, m, gin, gout):
        if isinstance(gout, (tuple,list)): gout = gout[0]
        self.gradients = gout

    def generate_cam(self, x):
        self.model.zero_grad()
        out = self.model(x)
        score = (out[:,0] + out[:,1]).sum()
        score.backward()

        if self.gradients is None or self.activations is None:
            B,C,H,W = x.shape
            return torch.zeros((B,1,H//32,W//32), device=x.device)

        w = torch.mean(self.gradients, dim=(2,3), keepdim=True)
        cam = torch.sum(w * self.activations, dim=1, keepdim=True)
        cam = F.relu(cam)
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        return cam.detach()


class StressTester:
    def __init__(self, model, device="cuda"):
        self.model = model.to(device).eval()
        self.device = device

        enc = model.encoder
        if hasattr(enc, "block4"):
            layer = enc.block4[-1]
        else:
            layer = list(enc.children())[-1]

        print(f"[{model.__class__.__name__}] Hooking Grad-CAM to: {layer}")
        self.gradcam = GradCAMGenerator(model, layer)

    def cleanup(self):
        self.gradcam.remove_hooks()

    # ------------------------------
    # Normalize Predictions
    # ------------------------------
    @torch.no_grad()
    def _normalize(self, pred):
        if torch.is_tensor(pred): pred = pred.cpu().numpy()
        a, b = pred[:,0], pred[:,1]
        d = a + b + 1e-8
        return np.stack([a/d, b/d], axis=1)

    # ------------------------------
    # ZOOM TEST
    # ------------------------------
    @torch.no_grad()
    def zoom_image(self, img, z):
        if z == 1.0: return img
        B,C,H,W = img.shape
        ch, cw = int(H/z), int(W/z)
        sh, sw = (H-ch)//2, (W-cw)//2
        crop = img[:,:,sh:sh+ch, sw:sw+cw]
        return F.interpolate(crop, (H,W), mode="bilinear", align_corners=False)

    def run_zoom(self, img, factors=[1.0, 1.5, 2.0]):
        img = img.to(self.device)
        res = {"factors": factors, "preds": [], "cams": []}

        with torch.enable_grad():
            base = self.gradcam.generate_cam(img)

        for f in factors:
            zm = self.zoom_image(img, f)
            pred = self.model(zm)
            res["preds"].append(self._normalize(pred))
            with torch.enable_grad():
                cam = self.gradcam.generate_cam(zm)
                cam = F.interpolate(cam, base.shape[2:], mode="bilinear")
                res["cams"].append(cam.cpu())

        res["base"] = base.cpu()
        return res

    # ------------------------------
    # RADIAL MASK TEST
    # ------------------------------
    @torch.no_grad()
    def mask_center_radial(self, image, radius):
        B, C, H, W = image.shape
        y, x = torch.meshgrid(
            torch.arange(H, device=image.device) - H//2,
            torch.arange(W, device=image.device) - W//2,
            indexing="ij"
        )
        R = torch.sqrt(x**2 + y**2)
        mask = (R >= radius).float()
        return image * mask.unsqueeze(0).unsqueeze(0)

    @torch.no_grad()
    def run_radial(self, img, radii=[0, 200, 400, 600]):
        img = img.to(self.device)
        res = {"radii": radii, "preds": []}
        for r in radii:
            masked = self.mask_center_radial(img, r)
            pred = self.model(masked)
            res["preds"].append(self._normalize(pred))
        res["preds"] = np.array(res["preds"])
        return res

    # ------------------------------
    # Visualization
    # ------------------------------
    def plot_zoom(self, img, res, title):
        factors = res["factors"]
        cams = res["cams"]
        base = res["base"]
        preds = res["preds"]

        plt.figure(figsize=(4*len(factors),8))
        for i,f in enumerate(factors):
            cam = cams[i].squeeze().numpy()
            img_np = img[0].cpu().numpy().transpose(1,2,0)
            if img_np.max()>1: img_np/=255
            cam = cv2.resize(cam, img_np.shape[:2][::-1])
            overlay = 0.5*img_np + 0.5*plt.cm.jet(cam)[:,:,:3]
            plt.subplot(2,len(factors),i+1)
            plt.imshow(overlay)
            plt.title(f"{f}x")
            plt.axis("off")

            plt.subplot(2,len(factors),len(factors)+i+1)
            a,b = preds[i][0]
            plt.bar(["α","β"], [a,b])
            plt.ylim(0,1)

        plt.suptitle(title)
        plt.show()

    def plot_radial(self, res, title):
        radii = res["radii"]
        preds = res["preds"].squeeze()

        plt.figure(figsize=(8,5))
        plt.plot(radii, preds[:,0], "-o", label="α")
        plt.plot(radii, preds[:,1], "-o", label="β")
        plt.title(title)
        plt.xlabel("Radius Mask (px)")
        plt.ylabel("Normalized Prediction")
        plt.legend()
        plt.grid()
        plt.show()

In [ ]:
dataset = load_dataset(cfg)

ckpt_imgnet = find_checkpoint(cfg["ckpt_imagenet"])
ckpt_synth  = find_checkpoint(cfg["ckpt_synth"])

print("Imagenet ckpt:", ckpt_imgnet)
print("Synthetic ckpt:", ckpt_synth)

model_imgnet = load_model(ckpt_imgnet)
model_synth  = load_model(ckpt_synth)

tester_imgnet = StressTester(model_imgnet, device)
tester_synth  = StressTester(model_synth, device)

In [ ]:
indices = np.random.choice(len(dataset), min(3,len(dataset)), replace=False)

for idx in indices:
    img, label = dataset[idx]
    img_batch = img.unsqueeze(0)

    print(f"\n===============================")
    print(f"Testing Image {idx}")
    print("===============================")

    # Zoom
    zoom_imgnet = tester_imgnet.run_zoom(img_batch)
    zoom_synth  = tester_synth.run_zoom(img_batch)

    # Radial
    radial_imgnet = tester_imgnet.run_radial(img_batch)
    radial_synth  = tester_synth.run_radial(img_batch)

    # Plot
    tester_imgnet.plot_zoom(img_batch, zoom_imgnet,
        f"Zoom Test — ImageNet — {cfg['name']} — idx {idx}")
    tester_synth.plot_zoom(img_batch, zoom_synth,
        f"Zoom Test — Synthetic — {cfg['name']} — idx {idx}")

    tester_imgnet.plot_radial(radial_imgnet,
        f"Radial Mask Test — ImageNet — {cfg['name']} — idx {idx}")
    tester_synth.plot_radial(radial_synth,
        f"Radial Mask Test — Synthetic — {cfg['name']} — idx {idx}")